In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import time
import datetime
import os
#from utils_laj import *
from data_processing import get_SensorData, my_pca, series_to_supervised, lstm_sampling
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor as GBR
from sklearn.metrics import mean_squared_error
import xgboost

today = datetime.date.today()

/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
path_checkpoint = './save/save_lstm/lstm_2_layers'
input_path = './input_data'
file = os.path.join(input_path,'sensors.csv')
target_file = os.path.join(input_path,'coke_target.csv')

In [3]:
# Считываем данные из файлов, нормализуем и применяем метод главных комполнентов
X, y, submit_X, mean_y, std_y = get_SensorData(file, target_file, nc=10)


In [4]:
print(mean_y, std_y)

3.159007111780772 0.5062311242891079


In [5]:
evals = pca.explained_variance_ratio_
evals_cs = evals.cumsum()
print(evals, evals_cs)
# Кумулятивное variance_explained равно 93%

[0.66839234 0.05001844 0.0454892  0.04043455 0.02760575 0.02430107
 0.02172642 0.01984493 0.01803035 0.01471829] [0.66839234 0.71841079 0.76389998 0.80433454 0.83194029 0.85624136
 0.87796778 0.89781272 0.91584307 0.93056136]


In [6]:
# Размерности датасета с известным целевым y и полного датасета
print("Data X: %s, y %s, submit_X %s" % (X.shape, y.shape, submit_X.shape))

Data X: (10400, 11), y (10400, 2), submit_X (13272, 11)


In [7]:
# преобразование датасета для возможности supervised learning
# используем для обучения и валидации модели данные за предыдущие 48 часов (time_length)
time_length = 200
Xl = series_to_supervised(X.iloc[:,1:len(list(X))], n_in=time_length, n_out=1)
yl = y['target'][time_length:]
print("Data Xl: %s, yl %s" % (Xl.shape, yl.shape))
# формируем данные конкурса по аналогичному принципу
Xs = series_to_supervised(submit_X.iloc[Xl.shape[0]:,1:len(list(submit_X))], n_in=time_length, n_out=1)

# выделим последние 25% в тестовый датасет
x_train, x_test, y_train, y_test = train_test_split(Xl, yl, test_size=0.25, random_state=42, shuffle=False)
print("Data x_train: %s, x_test: %s, y_train %s, y_test %s" % (x_train.shape, x_test.shape, y_train.shape, y_test.shape))

Data Xl: (10200, 2010), yl (10200,)
Data x_train: (7650, 2010), x_test: (2550, 2010), y_train (7650,), y_test (2550,)


In [11]:
filename = "mygbm2.pkl"
import pickle

In [13]:
my_gbm = pickle.load(open(filename, 'rb'))

In [14]:
y_pred = my_gbm.predict(x_test)


In [15]:
mse = mean_squared_error(y_test, y_pred)
print("MSE: %.4f" % mse)

MSE: 1.4341


In [16]:
my_gbm.loss_(y_test, y_pred)

1.4341383818932758

In [41]:
y_submit = my_gbm.predict(Xs)*std_y+mean_y
sub_file = pd.DataFrame()
sub_file["timestamp"] = submit_X["timestamp"][Xl.shape[0]+time_length:]
sub_file["target"] = y_submit
sub_file.to_csv('submit.csv', index=False)